In [1]:
import openai
from time import sleep

def load_api_key(key_file='../educational-prompt-engineering/key.txt'):
    with open(key_file) as f:
        # Get API key from text file
        key = f.read().strip("\n")
    return key

openai.api_key = load_api_key()

Example prompt:

In [2]:
def develop_prompt(grade):
    context = f'You are a grade {grade} student in an English class.'
    task = 'Write a paragraph on a topic assigned by your teacher for a purpose that is also assigned by your teacher.'
    response = ('First list the topic and purpose in the form: \n\n# Topic: {insert_topic_here}\n\n# Purpose: {insert_purpose_here}\n\n Then follow '+
                f'this with your paragraph. Write in the style and at the level that is appropriate for a {grade} student.')

    # Put it all together
    prompt = f'# CONTEXT #\n\n{context}\n\n# TASK #\n\n{task}\n\n# RESPONSE #\n\n{response}'

    return prompt
grade = 10
print(develop_prompt(grade=grade))

# CONTEXT #

You are a grade 10 student in an English class.

# TASK #

Write a paragraph on a topic assigned by your teacher for a purpose that is also assigned by your teacher.

# RESPONSE #

First list the topic and purpose in the form: 

# Topic: {insert_topic_here}

# Purpose: {insert_purpose_here}

 Then follow this with your paragraph. Write in the style and at the level that is appropriate for a 10 student.


In [3]:
model = 'gpt-4-turbo-preview' 
max_tokens = 1000

# Define the prompt
prompt = develop_prompt(grade)

messages = [{'role': 'system', 'content': 'You are a helpful assistant who excels at generating randomized paragraph topics, purposes, and text content.'},
            {'role': 'user', 'content': prompt}]

response = openai.ChatCompletion.create(model=model, messages=messages, max_tokens=max_tokens)

txt = response['choices'][0]['message']['content']
print(txt)

# Topic: The Importance of Recycling

# Purpose: To persuade

Recycling is not just an environmental fad; it's a crucial practice that benefits our planet in numerous ways. Every day, the earth is bombarded with waste - from plastic bottles to old electronics that pile up in landfills, posing a significant threat to our environment. Recycling these materials can greatly reduce the strain on our planet. It conserves natural resources, such as timber, water, and minerals, ensuring they last longer for future generations. Moreover, recycling saves energy by reducing the need to extract and process raw materials, which is often a heavy burden on our planet's resources. Additionally, it helps combat climate change by significantly lowering greenhouse gas emissions compared to new production processes. By recycling, we're not only making a positive impact on the environment but also promoting a sustainable lifestyle that can lead to a healthier planet. Consequently, understanding and partici

In [ ]:
import numpy as np
import h5py
num_students = 1000
model = 'gpt-3.5-turbo-1106' 
max_tokens = 1000

# Create a new HDF5 file
mode = 'a'
with h5py.File('data/grades_texts_new.h5', mode) as h5f:
    # Create a dataset for grades with maximum size
    if mode=='w':
        max_size = (None,)  # None indicates an unlimited dimension
        grade_dset = h5f.create_dataset('grade', shape=(0,), maxshape=max_size, dtype='i')
    
        # Create a special datatype for variable-length strings
        dt = h5py.special_dtype(vlen=str)
    
        # Create a dataset for texts with the special datatype and maximum size
        text_dset = h5f.create_dataset('text', shape=(0,), maxshape=max_size, dtype=dt)
    elif mode=='a':
        # Append to existing datafile
        grade_dset = h5f['grade']
        text_dset = h5f['text']

    # Being message thread
    if mode=='w':
        messages = [{'role': 'system', 'content': 'You are a helpful assistant who excels at generating randomized paragraph topics, purposes, and text content.'}]
    # Append data to datasets in a loop
    for i in range(num_students):
        grade = np.random.choice(np.arange(3,13))

        # Define the prompt
        prompt = develop_prompt(grade)
        messages.append({'role': 'user', 'content': prompt})

        # Prompt GPT with some randomness
        try:
            response = openai.ChatCompletion.create(model=model, messages=messages, max_tokens=max_tokens, 
                                                    seed=np.random.randint(0,1e5), temperature=np.random.uniform(0.7,1.3))
        except APIError:
            print('Server Error')
            sleep(10)
            continue
        # Obtain response
        text = response['choices'][0]['message']['content']

        # Save response as part of the conversation (or else it repeats its responses)
        messages.append({'role': 'assistant', 'content': text})
        # Only keep the last 5 back-and-forths
        if len(messages)>10:
            messages = [messages[0]] + messages[-10:]
        
        # Resize datasets to accommodate new data
        grade_dset.resize((i + 1,))
        text_dset.resize((i + 1,))

        # Assign data
        grade_dset[i] = grade
        text_dset[i] = text

In [26]:
with h5py.File('data/grades_texts_new.h5', 'r') as h5f:
    for i in range(20):
        print(f'\n# Grade: {h5f['grade'][i]}\n')
        print(h5f['text'][i].decode('utf-8'))


 3
# Topic: My Favorite Animal

# Purpose: To describe why it is my favorite

My favorite animal is a cat. I love cats because they are so fluffy and cute. They have soft fur that feels nice when I pet them, and their purring sounds make me happy. Cats are also very playful, and it's fun to watch them chase after balls or pounce on toys. Another reason I like cats is that they can be very friendly and like to cuddle. Sometimes, my cat jumps onto my lap when I am watching TV, and it makes me feel cozy and loved. That's why cats are my favorite animals!

 10
# Topic: The Impact of Global Warming

# Purpose: To persuade others to take action

Global warming presents a dire threat to the planet, demanding immediate action from everyone. This phenomenon, characterized by the gradual increase in Earth's average surface temperature, is primarily driven by human activities, such as the burning of fossil fuels, which release an excessive amount of greenhouse gases into the atmosphere. The cons